# Linear Regression Code Along

In [2]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop3.2')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mllr').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
df = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True,header=True)

In [6]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
df.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [11]:
for i in df.head(1)[0]:
    print(i)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [28]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol='features')

In [29]:
output = assembler.transform(df)

In [30]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [32]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [33]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [35]:
final_data = output.select('features','Yearly Amount Spent')

In [36]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [38]:
train_data,test_data = final_data.randomSplit([0.8, 0.2])

In [39]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                406|
|   mean|  499.3667467776187|
| stddev|  81.60374833119825|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [40]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                 94|
|   mean| 499.08638231470485|
| stddev|  68.95189974330656|
|    min|  319.9288698031936|
|    max|  663.8036932755083|
+-------+-------------------+



In [41]:
reg = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent')

In [42]:
reg_model = reg.fit(train_data)

22/05/24 12:33:16 WARN Instrumentation: [3908c2fc] regParam is zero, which might cause numerical instability and overfitting.
22/05/24 12:33:16 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/05/24 12:33:16 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/05/24 12:33:16 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [43]:
test_result = reg_model.evaluate(train_data)

In [44]:
test_result.residuals.show()

/home/ubuntu/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+
|           residuals|
+--------------------+
|   8.635525627722245|
| -5.5904970114477806|
|  -1.217182282171791|
|   9.488404684947795|
| -17.031975351644746|
|  -4.965778098251974|
|   5.341868811687846|
|  -7.423543682643242|
|  -14.08648870642736|
|  -8.826277222115323|
|   9.806779156060884|
| -1.2355408508614119|
|  20.513805230280468|
|  2.1909350502954794|
|   18.20088827995329|
|  3.0330369914585162|
|   3.001514482642392|
|-0.01464612084532746|
|  2.1517407648254334|
| -6.0611784164450455|
+--------------------+
only showing top 20 rows



In [46]:
test_result.rootMeanSquaredError

9.997435617498896

In [47]:
test_result.r2

0.9849537585494461

In [48]:
unlabel_data = test_data.select('features')

In [49]:
unlabel_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.8364326747734...|
|[31.0472221394875...|
|[31.1239743499119...|
|[31.2681042107507...|
|[31.4252268808548...|
|[31.4474464941278...|
|[31.7656188210424...|
|[31.8164283341993...|
|[31.8186165667690...|
|[31.8279790554652...|
|[31.9563005605233...|
|[31.9764800614612...|
|[32.0123007682454...|
|[32.0478146331398...|
|[32.0705462209254...|
|[32.1878120459321...|
|[32.2129238300579...|
|[32.2272991363672...|
|[32.2599732655570...|
+--------------------+
only showing top 20 rows



In [50]:
prediction = reg_model.transform(unlabel_data)

In [51]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|332.31347536497515|
|[30.8364326747734...| 472.8881254943958|
|[31.0472221394875...|388.52936061779315|
|[31.1239743499119...|  509.848842518491|
|[31.2681042107507...| 428.1590371032919|
|[31.4252268808548...| 535.7918789028442|
|[31.4474464941278...| 427.1077126480693|
|[31.7656188210424...| 501.8878513151458|
|[31.8164283341993...| 519.3552637263504|
|[31.8186165667690...|449.38989516771903|
|[31.8279790554652...|  449.659924981062|
|[31.9563005605233...| 565.9822470811207|
|[31.9764800614612...|325.19044842505264|
|[32.0123007682454...|489.80738831259555|
|[32.0478146331398...|481.39678705156643|
|[32.0705462209254...| 532.7731872026707|
|[32.1878120459321...| 456.0213619156175|
|[32.2129238300579...| 514.6807992009499|
|[32.2272991363672...| 622.4103096473048|
|[32.2599732655570...| 574.0688888722143|
+--------------------+------------